# Download STIX data using stixpy

[17-Jul-2023] At the time being, this notebook work with the REPROJ notebook

### Imports and functions

In [1]:
import numpy as np
import os
import pandas as pd

from sunpy.net import Fido, attrs as a
from stixpy.net.client import STIXClient

In [2]:
def get_cpd_data(tstart, tend, request_id, folder_store = ""):
    '''
    Download the STIX cpd from the website
    '''
    
    # Get the folder delimiter depending on the OS
    if os.name == "nt":
        folder_delimiter = "\\"
    else:
        folder_delimiter = "/"

    # If folder_store is given, check if it is effectively a folder
    if folder_store != "":
        if folder_store[-1] != folder_delimiter:
            folder_store += folder_delimiter

    # Query the files
    res = Fido.search(a.Time(tstart, tend), a.Instrument.stix, a.stix.DataProduct.sci_xray_cpd)
    
    # Find the file matching the UID given as input
    list_bool = list(res["stix"]["Request ID"] == int(this_uid))
    idx = [i for i, x in enumerate(list_bool) if x]
    #res = res["stix"][list(res["stix"]["Request ID"]) == request_id]
    res = res["stix"][idx]
    
    # Download the file
    file = Fido.fetch(res, path=folder_store+"{file}")

    # Return the path to the file
    if len(file)>0:
        return folder_store+file[0]
    else:
        return ''

In [3]:
def get_aux_data(tstart, tend, folder_store = ""):
    '''
    Download the STIX cpd from the website
    '''

    # Get the folder delimiter depending on the OS
    if os.name == "nt":
        folder_delimiter = "\\"
    else:
        folder_delimiter = "/"

    # If folder_store is given, check if it is effectively a folder
    if folder_store != "":
        if folder_store[-1] != folder_delimiter:
            folder_store += folder_delimiter

    # Query the files
    res = Fido.search(a.Time(tstart, tend), a.Instrument.stix, a.stix.DataProduct.aux_ephemeris)

    # Download the file
    file = Fido.fetch(res, path=folder_store+"{file}")

    # Return the path to the file
    if len(file)>0:
        return folder_store+file[0]
    else:
        return ''

### Parameters

In [4]:
# Folder where to store the STIX data
folder_store = '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/Data/SO_STIX/'

# Path to the Frederic's table
frederic_csv = '/home/afbattaglia/Software/STIX_estimate_pointing_error/Flare-location_test-table.csv'

### Main code

In [5]:
# Open Frederic's table to get the flare times and UIDs
df = pd.read_csv(frederic_csv)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Time range (UTC @ SolO),Unnamed: 5,Energy range [keV],Unnamed: 7
0,Peak time (UTC),GOES,SSW loc.,UID,t min,t max,E min,E max
1,2021-05-07T18:50,M3.9,N17E78,2105070034,2021-05-07T18:50:00,2021-05-07T19:09:00,15,28
2,2021-09-23T15:24,M1.9,S29E16,2109230075,2021-09-23T15:21:18,2021-09-23T15:21:40,35,60
3,2021-10-09T06:28,M1.6,N18E08,2110090002,2021-10-09T06:27:32,2021-10-09T06:29:49,30,80
4,2022-01-20T05:58,M5.5,N08W76,2201208962,2022-01-20T05:54:53,2022-01-20T05:55:41,25,45
5,2022-03-02T17:35,M2.0,N15E29,2203026385,2022-03-02T17:34:15,2022-03-02T17:34:40,15,28
6,2022-03-11T22:26,M2.3,S27W63,2203118542,2022-03-11T22:24:15,2022-03-11T22:24:40,18,28
7,2022-03-14T08:34,M2.0,N23E03,2203143075,2022-03-14T08:33:50,2022-03-14T08:34:23,18,28
8,2022-03-15T12:33,M1.5,N23W10,2203159015,2022-03-15T12:31:05,2022-03-15T12:33:25,18,28
9,2022-03-15T22:41,M1.6,N23W17,2203159363,2022-03-15T22:38:24,2022-03-15T22:39:24,25,50


In [7]:
# Store variables of interests
uid = list(df.iloc[1:,3])
peak_time_so = list(df.iloc[1:,0])
start_time_so = list(df.iloc[1:,4])
end_time_so = list(df.iloc[1:,5])
len(uid), uid

(16,
 ['2105070034',
  '2109230075',
  '2110090002',
  '2201208962',
  '2203026385',
  '2203118542',
  '2203143075',
  '2203159015',
  '2203159363',
  '2203282437',
  '2203293826',
  '2204023786',
  '2204176813',
  '2204185406',
  '2204184714',
  '2208271589'])

In [8]:
# Get the folder delimiter depending on the OS
if os.name == "nt":
    folder_delimiter = "\\"
else:
    folder_delimiter = "/"

# If folder_store is given, check if it is effectively a folder
if folder_store != "":
    if folder_store[-1] != folder_delimiter:
        folder_store += folder_delimiter

In [9]:
# Loop on all elements in the list
aux_paths = []
cpd_paths = []
for i in range(len(uid)):

    this_uid = uid[i]
    this_peak_time = peak_time_so[i]
    this_start_time = start_time_so[i]
    this_end_time = end_time_so[i]

    print(this_peak_time)

    # If is not NaN
    if this_uid != np.nan:

        # Create a new folder containing the UID and peak time as filename
        fname_folder = folder_store + this_peak_time + "_" + this_uid + folder_delimiter
        if not os.path.exists(fname_folder):
            os.makedirs(fname_folder)

        cpd_paths.append(get_cpd_data(this_start_time, this_end_time, this_uid, folder_store = fname_folder))
        aux_paths.append(get_aux_data(this_start_time, this_end_time, folder_store = fname_folder))

2021-05-07T18:50


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

2021-09-23T15:24


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

...

2022-08-27T02:19


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

solo_L1_stix-sci-xray-cpd_20220827T013602-20220827T022843_V01_2208271589-62609.fits:   0%|          | 0.00/25.…

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

solo_L2_stix-aux-ephemeris_20220827_V01.fits:   0%|          | 0.00/397k [00:00<?, ?B/s]